importation des librairie

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio    
import tempfile
import argparse
import gc
from shapely.geometry import shape
gc.collect()
import geopandas as gpd
import gcsfs
import rasterio
import numpy as np
import geopandas as gpd
import geopandas as gpd
from shapely.ops import substring
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.enums import Resampling


parametre

In [ ]:
sizesplit = 100  # size to split roads
pathflood="CoCliCo_data/DCS1_TWL5.tif"# PATH to the coastal flooding raster file, The file must be in .tif format (GeoTIFF).

fonction

In [ ]:

def split_line(line, max_length=500): 
    if line.length <= max_length:
        return [line]
    segments = []
    start = 0.0
    while start < line.length:
        end = min(start + max_length, line.length)
        segment = substring(line, start, end)  
        segments.append(segment)
        start = end
    return segments

def split2(gdf, max_length=500): 
    new_geometries = []
    new_attributes = []
    for _, row in gdf.iterrows():
        line = row['geometry']
        segments = split_line(line, max_length=max_length)
        for segment in segments:
            new_geometries.append(segment)
            new_attributes.append(row.drop('geometry'))
    return gpd.GeoDataFrame(new_attributes, geometry=new_geometries, crs=gdf.crs)


def modify_raster(input_raster_path): 
    with rasterio.open(input_raster_path) as src:
        raster_data = src.read(1) 
        nodata_value = src.nodata
        modified_data = np.full(raster_data.shape, np.nan, dtype=rasterio.float32)
        if nodata_value is not None:
            valid_mask = raster_data != nodata_value
            modified_data[valid_mask] = 1
        else:
            valid_mask = ~np.isnan(raster_data)
            modified_data[valid_mask] = 1
        return modified_data


def raster_value_to_shapefile(raster_path, target_value=1):
    with rasterio.open(raster_path) as src:
        raster_data = src.read(1)
        transform = src.transform
        crs = src.crs
        nodata = src.nodata
        shapes_and_values = rasterio.features.shapes(raster_data, transform=transform)
        geometries = []
        for geom, value in shapes_and_values:
            if value == target_value:  
                geometries.append({
                    'geometry': shape(geom),
                    'properties': {'value': value}
                })
        
        gdf = gpd.GeoDataFrame(geometries, crs=crs)
        return gdf
    
def reproject_raster(input_raster, target_crs="EPSG:3035"):
    with rasterio.open(input_raster) as src:
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        output_raster = input_raster.replace(".tif", "_3035.tif")
        with rasterio.open(output_raster, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest
                )
        return output_raster


def save_modified_raster(modified_data, transform, crs, output_path):
    with rasterio.open(output_path, 'w', driver='GTiff', 
                       height=modified_data.shape[0], width=modified_data.shape[1],
                       count=1, dtype=modified_data.dtype, crs=crs, transform=transform) as dst:
        dst.write(modified_data, 1)


        
        



liens des fichiers

In [ ]:

path = "gs://coclico-data-public/coclico/LAU/LAU_2020_NUTS_2021_01M_3035.parquet"
local_path = "CoCliCo_data/LAU.parquet"
fs = gcsfs.GCSFileSystem()
fs.get(path, local_path)

unitgeo="CoCliCo_data/LAU.parquet"
lau=gpd.read_parquet(unitgeo)
pathflood= reproject_raster(pathflood)
flood = modify_raster(pathflood)#
with tempfile.NamedTemporaryFile(delete=False, suffix=".tif") as tmpfile:
    temp_raster_path = tmpfile.name
    with rasterio.open(pathflood) as src:
        save_modified_raster(flood, src.transform, src.crs, temp_raster_path) 
    floodshp = raster_value_to_shapefile(temp_raster_path)
intersection= gpd.overlay(lau, floodshp, how="intersection")
unique_values = intersection['nuts_2'].unique()

In [ ]:
for i in range (len(unique_values)):
    path = "gs://coclico-data-public/coclico/ceed/"
    path=path+unique_values[i]+"_CEED.parquet"
    local_path = "CoCliCo_data/CEED/"+unique_values[i]+"_CEED.parquet"
    fs = gcsfs.GCSFileSystem()
    fs.get(path, local_path)


code

In [ ]:
stock=gpd.GeoDataFrame()

for i in range (len(unique_values)):
    local_path = "CoCliCo_data/CEED/"+unique_values[i]+"_CEED.parquet"
    line=gpd.read_parquet(local_path)
    line.reset_index(inplace=True)             
    line=line[line["geometry"].geom_type=="LineString"]
    
    line=line[line["level_0"]=="road"]
    line.drop(columns=['level_0'], inplace=True)
    line=gpd.sjoin(line,intersection,how="inner",predicate="intersects")
    (b,d)=line.shape
    if b>0:
        line=split2(line,sizesplit)
        line["surfacecal"]=line['geometry'].length
        for df_ in [line, intersection]:
            if 'index_right' in df_.columns:
                df_.drop(columns='index_right', inplace=True)
        line=gpd.sjoin(line,intersection,how="inner",predicate="intersects")
        a = line.groupby('FID_right')['surfacecal'].sum().reset_index(name='length')
        stock=gpd.GeoDataFrame(pd.concat([stock, a], ignore_index=True))
stock=stock[["FID_right","length"]]        
print(stock)